# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
file = ("Resources/weather_all.csv")

weather_all_df = pd.read_csv(file)
weather_all_df



,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Guerrero Negro,27.9769,-114.0611,67.73,64,0,14.29,MX,1635091451
1,Atuona,-9.8000,-139.0333,77.81,74,19,18.10,PF,1635091451
2,Mataura,-46.1927,168.8643,49.24,98,91,0.54,NZ,1635091451
3,Nicosia,35.1667,33.3667,71.08,56,20,12.66,CY,1635091451
4,Okato,-39.2000,173.8833,53.69,94,100,8.52,NZ,1635091452
...,...,...,...,...,...,...,...,...,...
572,Lastoursville,-0.8143,12.7287,78.96,89,62,0.16,GA,1635091635
573,Wewak,-3.5534,143.6268,79.18,83,100,4.54,PG,1635091635
574,Krasnyy Oktyabr',56.1167,38.8833,33.76,77,98,12.53,RU,1635091365
575,Morwa,22.9000,73.8333,75.45,57,40,7.34,IN,1635091636


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_all_df[["Lat", "Lng"]].astype(float)
humidity = weather_all_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [114]:
perfect_weather_df = weather_all_df

perfect_weather_df = perfect_weather_df.loc[(perfect_weather_df["Temp"] < 95) & (perfect_weather_df["Temp"] > 70)]
perfect_weather_df = perfect_weather_df.loc[perfect_weather_df["Wind Speed"] < 10]
perfect_weather_df = perfect_weather_df.loc[perfect_weather_df["Cloudiness"] <= 30]
perfect_weather_df = perfect_weather_df.loc[perfect_weather_df["Humidity"] <=30]

perfect_weather_df

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,Najrān,17.4924,44.1277,73.38,16,0,7.65,SA,1635091477
400,Ibrā’,22.6906,58.5334,87.13,23,18,6.62,OM,1635091580
404,Abu Kamal,34.4506,40.9171,75.18,26,0,5.17,SY,1635091581
424,Bilma,18.6853,12.9164,94.06,9,0,6.35,NE,1635091586
433,Tsabong,-26.0217,22.4010,78.71,16,0,6.26,BW,1635091589
550,Dutse,11.7594,9.3392,92.34,23,15,5.64,NG,1635091628
552,Riyadh,24.6877,46.7219,91.54,13,0,8.66,SA,1635091524
567,Aranos,-24.1333,19.1167,82.02,13,0,2.35,NaN,1635091633


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [115]:
hotel_df = pd.DataFrame (perfect_weather_df)
hotel_df

hotel_df["Hotel"]= ""

params = {
    "radius" : 5000,
    "types" : "lodging",
    "key" : g_key}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")





Retrieving Results for Index 95: Najrān.
Closest hotel is محمد هادي ال فطيح.
------------
Retrieving Results for Index 400: Ibrā’.
Closest hotel is مزرعة راحة النفوس المنزفة.
------------
Retrieving Results for Index 404: Abu Kamal.
Closest hotel is محطة التلفاز.
------------
Retrieving Results for Index 424: Bilma.
Closest hotel is Expeditions Ténére voyages.
------------
Retrieving Results for Index 433: Tsabong.
Closest hotel is Springbok Motel.
------------
Retrieving Results for Index 550: Dutse.
Closest hotel is Central Mosque Galamawa.
------------
Retrieving Results for Index 552: Riyadh.
Closest hotel is InterContinental Riyadh, an IHG Hotel.
------------
Retrieving Results for Index 567: Aranos.
Closest hotel is Ley-Lia Guest House.
------------


In [116]:
hotel_df.drop("Hotel", axis =1)

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
95,Najrān,17.4924,44.1277,73.38,16,0,7.65,SA,1635091477,محمد هادي ال فطيح
400,Ibrā’,22.6906,58.5334,87.13,23,18,6.62,OM,1635091580,مزرعة راحة النفوس المنزفة
404,Abu Kamal,34.4506,40.9171,75.18,26,0,5.17,SY,1635091581,محطة التلفاز
424,Bilma,18.6853,12.9164,94.06,9,0,6.35,NE,1635091586,Expeditions Ténére voyages
433,Tsabong,-26.0217,22.4010,78.71,16,0,6.26,BW,1635091589,Springbok Motel
550,Dutse,11.7594,9.3392,92.34,23,15,5.64,NG,1635091628,Central Mosque Galamawa
552,Riyadh,24.6877,46.7219,91.54,13,0,8.66,SA,1635091524,"InterContinental Riyadh, an IHG Hotel"
567,Aranos,-24.1333,19.1167,82.02,13,0,2.35,NaN,1635091633,Ley-Lia Guest House


In [117]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [118]:
# Add marker layer ontop of heat map
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig



Figure(layout=FigureLayout(height='420px'))